In [139]:
import polars as pl
import json
from datetime import date
from transform import create_table_song

In [5]:
pl.Config.set_tbl_cols(20)
pl.Config.set_tbl_rows(100)
pl.Config.set_fmt_str_lengths(70)

polars.config.Config

In [141]:
def clean(lf):
    return (
        lf.cast({"date": pl.Date, "position": pl.UInt8})
        .sort(by="date")
        .with_row_index("_id")
        .select(["_id", "date", "position", "song", "artist"])
    )

In [26]:
lf0: pl.LazyFrame = clean(pl.read_json("../data/records05-29_23-57.json").lazy())

In [27]:
def handle_edge_cases(lf):
    edge_pat_1 = r"(?i)duet with"
    edge_pat_2 = r"(?i)\((feat\.*[a-z]*)|(&)|(with)"
    edge_pat_3: str = r"(?i)&\s(the|his|her|original)(.*)"
    return lf.unique(subset=["song", "artist"]).with_columns(
        artist=pl.col("artist").replace(edge_pat_1, "&")
    ).with_columns(
        artist=pl.when(pl.col("artist").str.contains(edge_pat_2))
        .then(pl.col("artist").str.replace_all(r"[()]", ""))
        .otherwise(pl.col("artist"))
    ).with_columns(
        artist=pl.col("artist").str.replace_all(edge_pat_3, r"and $1$2")
    )

In [81]:
def split_features(lf: pl.LazyFrame):
    split_pattern: str = r"(?i)(\sfeat\.*[a-z]*\s)|(\swith\s)"
    roles = ["main", "featured"]
    sep: str = "-"
    mainlf = lf.with_columns(
        artist=pl.col("artist")
        .str.replace(split_pattern, sep)
        .str.split_exact(sep, 1)
        .struct[0],
        role=pl.lit("main"),
    ).drop_nulls()
    featlf = lf.with_columns(
        artist=pl.col("artist")
        .str.replace(split_pattern, sep)
        .str.split_exact(sep, 1)
        .struct[1],
        role=pl.lit("featured"),
    ).drop_nulls()
    junction_lf = pl.concat([mainlf, featlf]).cast({"role": pl.Enum(roles)})
    return junction_lf

In [88]:
def split_artists(lf: pl.LazyFrame):
    seperator_sub: str = r"(?i)(\s*[&/+,]\s*)|(x\s)"
    sep: str = "!~!"
    transformed_lf = lf.with_columns(
        artist=pl.col("artist")
        .str.replace_all(seperator_sub, sep)
        .str.split(sep).list.eval(pl.element().str.strip_chars()),
    ).explode(["artist"])
    return transformed_lf

In [49]:
lf1 = handle_edge_cases(lf0)

In [50]:
lf2 = split_features(lf1)

In [71]:
lf3 = split_artists(lf2)

In [72]:
lf3.filter(
    pl.col("song") == "Dance With Me"
).collect()

song,artist,role
str,str,enum
"""Dance With Me""","""The Mojo Men""","""main"""
"""Dance With Me""","""Orleans""","""main"""
"""Dance With Me""","""Peter Brown""","""main"""
"""Dance With Me""","""112""","""main"""
"""Dance With Me""","""Debelah Morgan""","""main"""
"""Dance With Me""","""The Drifters""","""main"""
"""Dance With Me""","""Betty Wright""","""featured"""


In [142]:
song_df0 = create_table_song(lf0)

In [143]:
song_df1 = handle_edge_cases(song_df0.lazy())

In [144]:
song_df2 = split_features(song_df1)

In [145]:
song_df3 = split_artists(song_df2)

In [ ]:
song_df3.collect()

In [ ]:
song_df3.filter(
    pl.col("song") == "Dance With Me"
).collect()

In [146]:
artist_df0 = song_df3.sort("latest", descending=True).unique(subset=["artist"]).select("artist")

In [147]:
artist_df1 = artist_df0.with_row_index("id").sort("id", descending=True)

In [154]:
junction_lf = song_df3.join(artist_df1, on="artist", how="inner", suffix="artist")

In [ ]:
junction_lf = junction_lf.select(
        id_song="id",
        idartist="id_artist",
        role="contribution_artist",
        power="power_song",
        longevity="longevity_artist",
        decade="decade_song",
)

In [153]:
junction_lf.collect()

ColumnNotFoundError: id_song

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'sink' <---
INNER JOIN:
LEFT PLAN ON: [col("artist")]
  EXPLODE
     WITH_COLUMNS:
     [col("artist").str.replace(["(?i)(\s*[&/+,]\s*)|(x\s)", "!~!"]).str.split(["!~!"]).eval().alias("artist")] 
       WITH_COLUMNS:
       [col("role").strict_cast(Enum(Some(local), Physical))] 
        UNION
          PLAN 0:
            FILTER col("role").is_not_null()
            FROM
              FILTER col("decade").is_not_null()
              FROM
                FILTER col("latest").is_not_null()
                FROM
                  FILTER col("earliest").is_not_null()
                  FROM
                    FILTER col("longevity").is_not_null()
                    FROM
                      FILTER col("power").is_not_null()
                      FROM
                        FILTER col("artist").is_not_null()
                        FROM
                          FILTER col("song").is_not_null()
       